In [1]:
import camelot
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Helper codes

def clean_country_name(name):
    name = str(name).strip()              
    name = re.sub(r'\s+', ' ', name)     
    name = re.sub(r"\s*-\s*.*", "", name)
    name = re.sub(r"[^A-Za-z\s]", "", name)
    name = re.sub(r'[^A-Za-z\s]', '', name)  
    return name.title()                   

# --- Constants ---
MONTH_ORDER = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# --- Helpers ---
def get_months(df):
    """Return ordered list of existing month columns."""
    return [m for m in MONTH_ORDER if m in df.columns]


In [3]:
exclude_words = [
    "A S I A",
    "ASEAN",
    "SUB-TOTAL",
    "EAST ASIA",
    "SOUTH ASIA",
    "MIDDLE EAST",
    "A M E R I C A",
    "NORTH AMERICA",
    "SOUTH AMERICA",
    "E U R O P E",
    "WESTERN EUROPE",
    "NORTHERN EUROPE",
    "SOUTHERN EUROPE",
    "EASTERN EUROPE",
    "EASTERN MEDITERRANEAN EUROPE",
    "EASTERN MEDITERRANEAN EUROPE**"
    "AUSTRALASIA/PACIFIC",
    "TOTAL (CIS & RUSSIA)"
    "A F R I C A	"
]

In [4]:
# Read tables from PDF (PAGE 1)
pdf_file = "/Users/kim/Desktop/repos/Philippines_Visitor/data/rawData/2019-JAN-DEC.pdf"
tables = camelot.read_pdf(pdf_file, pages='1', flavor="stream")

# Combine all extracted tables into one DataFrame
df1 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df1 = df1.iloc[2:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df1.columns = range(df1.shape[1])

df1.drop(
    df1[df1[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

df1 = df1.iloc[:-1]
df1 = df1.reset_index(drop=True)


In [5]:
# Read tables from PDF (PAGE 2)
tables = camelot.read_pdf(pdf_file, pages='2', flavor="stream")

# Combine all extracted tables into one DataFrame
df2 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df2 = df2.iloc[3:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df2.columns = range(df2.shape[1])

df2.drop(
   df2[df2[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

# df2 = df2.drop(columns=[4,5,7,15])
# df2.columns = range(df2.shape[1])
df2 = df2[df2[0] != 'EASTERN MEDITERRANEAN EUROPE**'].reset_index(drop=True)

df2 = df2.reset_index(drop=True)
df2 = df2.iloc[:-1]


# DROP TOTAL RUSSIAN FEDERATION 
df2 = df2.reset_index(drop=True)
df2 = df2.drop(26).reset_index(drop=True)
df2 = df2.drop(23).reset_index(drop=True)

df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,ARGENTINA,571,400,371,358,303,178,156,177,197,253,349,330,"3,643",0.04,"3,468",5.05
1,BRAZIL,"1,047",909,"1,020",960,766,547,450,729,717,734,855,"1,301","10,035",0.12,"7,364",36.27
2,COLOMBIA,298,253,292,302,268,241,256,250,289,256,381,300,"3,386",0.04,"2,389",41.73
3,PERU,144,125,120,134,150,110,99,180,129,139,153,177,"1,660",0.02,"1,038",59.92
4,VENEZUELA,59,57,65,67,52,40,43,68,59,57,70,65,702,0.01,423,65.96
5,ANDORRA,,1,2,-,2,2,-,3,1,-,3,11,25,0.00,273,-90.84
6,AUSTRIA,"1,761","1,712","1,390","1,176","1,019",875,"1,284",843,724,984,"1,130","1,942","14,840",0.18,"14,193",4.56
7,BELGIUM,"1,585","1,513","1,699","1,967","1,271","1,523","2,316","1,175","1,060","1,320","1,642","2,085","19,156",0.23,"17,285",10.82
8,FRANCE,"9,285","12,934","10,714","10,380","5,882","3,765","5,564","5,638","3,903","5,640","6,999","7,873","88,577",1.07,"74,400",19.06
9,GERMANY,"11,081","11,569","11,075","9,485","6,917","5,195","6,857","6,512","5,732","7,698","8,862","12,773","103,756",1.26,"92,098",12.66


In [6]:
# Read tables from PDF (PAGE 3)
tables = camelot.read_pdf(pdf_file, pages='3', flavor="stream")

# Combine all extracted tables into one DataFrame
df3 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df3 = df3.iloc[3:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df3.columns = range(df3.shape[1])

df3.drop(
   df3[df3[0].str.upper().isin(exclude_words)].index,
    inplace=True
)
df3 = df3.drop([6,10]).reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df3 = df3.iloc[:-19]
df3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,AUSTRALIA,"29,929","21,571","21,647","28,683","22,021","19,163","21,364","17,347","21,346","23,096","23,068","36,935","286,170",3.46,"279,828",2.27
1,GUAM,"1,806","1,850","1,801","1,401","1,680","2,367","2,235","1,226",818,"1,540","1,667","1,444","19,835",0.24,"32,357",-38.70
2,NAURU,1,6,33,2,11,8,2,5,4,3,17,1,93,0.00,38,144.74
3,NEW ZEALAND,"3,496","2,300","2,627","3,770","3,026","2,782","3,193","2,488","2,870","3,189","3,257","4,874","37,872",0.46,"33,341",13.59
4,PAPUA NEW GUINEA,908,529,677,647,707,875,785,690,829,945,685,551,"8,828",0.11,"8,481",4.09
5,NIGERIA,268,243,255,254,519,285,279,297,340,220,291,188,"3,439",0.04,"2,104",63.45
6,SOUTH AFRICA,705,697,710,765,552,561,571,597,585,768,652,"1,390","8,553",0.10,"7,543",13.39
7,RESIDENCES,"9,055","9,306","8,615","7,014","5,648","5,814","6,214","5,381","5,532","6,217","9,155","8,848","86,799",1.05,"71,007",22.24
8,T O T A L,"718,118","762,437","709,399","653,336","612,861","638,440","712,285","699,933","604,552","634,786","679,273","763,057","8,188,477",99.12,"7,096,594",15.39
9,OVERSEAS FILIPINOS***,"5,272","4,428","4,910","9,651","8,858","5,340","6,772","2,910","2,001","3,763","4,790","13,741","72,436",0.88,"71,873",0.78


In [7]:
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

combined_df.columns = [
        "Country",
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
        "Total",
        "Percentage",
        "Previous Total",
        "Growth Rate"
        ]

# Clean and convert column to float
combined_df['Total'] = (
    combined_df['Total']
    .astype(str)               # ensure everything is string
    .str.replace(',', '')      # remove commas
    .str.strip()               # remove leading/trailing spaces
    .replace({'': np.nan, '-': np.nan})  # replace empty strings and '-' with NaN
    .astype(float)             # convert to float
)
combined_df = combined_df.sort_values(by="Total", ascending=False).reset_index(drop=True)

# Copy the row you want to move
row_to_move = combined_df.iloc[16]

# Drop the original row
combined_df = combined_df.drop(16).reset_index(drop=True)

top = combined_df.iloc[:1]   
bottom = combined_df.iloc[1:] 

# Insert the row
combined_df = pd.concat([top, pd.DataFrame([row_to_move]), bottom]).reset_index(drop=True)

combined_df['Country'] = combined_df['Country'].apply(clean_country_name)
combined_df['Country'] = combined_df['Country'].replace('Independent States', 'Commonwealth of Independent States')
combined_df['Country'] = combined_df['Country'].replace('Hongkong', 'Hong Kong')
combined_df['Country'] = combined_df['Country'].replace('Korea', 'South Korea')

combined_df.to_csv("/Users/kim/Desktop/repos/Philippines_Visitor/data/cleanedData/2019cleanData.csv")

In [8]:
# Store Monthly Data
monthly_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/monthly_visitors"

monthly_df = combined_df.iloc[2:].reset_index(drop=True) 
monthly_df['Country'] = monthly_df['Country'].apply(clean_country_name)

months = get_months(combined_df)

monthly_df = monthly_df[['Country'] + months]

mon_out_path = os.path.join(monthly_folder, f"2019_monthly.csv")
monthly_df.to_csv(mon_out_path, index=False)

print(f"Saved 2019 Monthly Visitors csv file")
monthly_df

Saved 2019 Monthly Visitors csv file


,Country,January,February,March,April,May,June,July,August,September,October,November,December
0,South Korea,"179,900","180,890","158,794","130,707","138,239","157,495","167,249","185,334","152,184","158,380","176,185","203,965"
1,China,"131,196","174,175","158,433","139,177","130,788","132,792","171,848","170,903","150,505","139,707","126,785","117,000"
2,Usa,"109,342","94,632","89,806","87,710","90,979","96,735","93,912","68,418","61,085","79,716","82,617","109,488"
3,Japan,"47,567","64,365","65,837","57,724","46,495","44,919","55,727","78,699","56,878","51,414","53,784","59,379"
4,Taiwan,"21,804","30,648","25,456","25,441","25,637","30,546","32,663","32,676","27,707","29,642","23,350","21,703"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Laos,72,61,108,91,90,102,144,113,131,160,280,102
64,Luxembourg,72,92,58,87,43,31,66,104,37,63,69,115
65,Venezuela,59,57,65,67,52,40,43,68,59,57,70,65
66,Nauru,1,6,33,2,11,8,2,5,4,3,17,1


In [9]:
# Store Special Categories Data

cat_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/special_category"

cat_df = combined_df.head(2)
cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
cat_df["Country"] = cat_df["Country"].astype(str).str.strip().str.title()
cat_df["Country"] = cat_df["Country"].apply(clean_country_name)

cat_df = cat_df.iloc[:, :-4]

cat_out_path = os.path.join(cat_folder, f"2019_category.csv")
cat_df.to_csv(cat_out_path, index=False)

print(f"Saved 2019 Category csv file")


Saved 2019 Category csv file


/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_44662/3424148744.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_44662/3424148744.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_44662/3424148744.py:8: SettingWithCo

In [10]:
year_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/yearly_total"

year_df = combined_df[['Country', 'Total', 'Percentage', 'Previous Total', 'Growth Rate']]
year_df = year_df.iloc[2:]

year_df["Previous Total"] = (
    year_df["Previous Total"]
        .astype(str)
        .str.replace('"', '', regex=False)
        .str.replace(',', '', regex=False)   # REMOVE COMMAS
        .str.strip()
)

year_df["Previous Total"] = year_df["Previous Total"].astype(float)

year_df["Growth Rate"] = pd.to_numeric(year_df["Growth Rate"], errors = 'coerce')
year_df["Percentage"] = pd.to_numeric(year_df["Percentage"], errors = 'coerce')

year_path = os.path.join(year_folder, f"2019_year.csv")
year_df.to_csv(year_path, index=False)

print(f"Saved 2019 Year csv file")

Saved 2019 Year csv file
